In [59]:
import numpy as np
import pandas as pd 
from scipy import stats
import googlemaps

In [42]:
train =  pd.read_csv("train_.csv",index_col="tripid")
test =  pd.read_csv("test_.csv",index_col="tripid")

In [44]:
# This distance function was not used in the final solution
def distance(s_lat, s_lng, e_lat, e_lng):
    
    # approximate radius of earth in km
    R = 6373.0
    
    s_lat = s_lat*np.pi/180.0                      
    s_lng = np.deg2rad(s_lng)     
    e_lat = np.deg2rad(e_lat)                       
    e_lng = np.deg2rad(e_lng)  
    
    d = np.sin((e_lat - s_lat)/2)**2 + np.cos(s_lat)*np.cos(e_lat) * np.sin((e_lng - s_lng)/2)**2
    
    return 2 * R * np.arcsin(np.sqrt(d))

# This distance function was not used in the final solution
def eucledian_distance(s_lat, s_lng, e_lat, e_lng):
    
    x2 = np.square(e_lat - s_lat)
    y2 = np.square(e_lng - s_lng)
    
    return np.sqrt(x2 + y2) * 100

# Getting real distances from google maps API
def getDistances(APIkey,df):
    gmaps = googlemaps.Client(key=API_key)

    distances = []

    for row in df.itertuples():
        start = (row.pick_lat,row.pick_lon)
        end = (row.drop_lat,row.drop_lon)

        dist = gmaps.distance_matrix(origins,
                                     destination, 
                                     mode='driving')["rows"][0]["elements"][0]["distance"]["value"]
        distances.append(dist)

    return pd.DataFrame(distances)


In [289]:
# Get real distances
train["distance"] = getDistance("APIkey",train)
test["distance"] = getDistance("APIkey",test)

In [31]:
# Outlier detection and removal
z_dist = np.abs(stats.zscore(train['distance']))
train = train[z_dist < 3]

In [45]:
# Drop NAN
train = train.dropna()

In [46]:
train["label"] = train["label"].apply(lambda x: int(x=="correct"))

In [47]:
y_train = train["label"]
X_train = train.loc[:,train.columns != 'label']
X_train.head(50)

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,distance,fare
tripid,,,,,,,,,,,,,
189123628,10.5,834.0,56.0,0.000000,64.0,11/1/2019 0:20,11/1/2019 0:34,6.86252,79.8993,6.90330,79.8783,7.801,270.32
189125358,10.5,791.0,47.0,0.000000,134.0,11/1/2019 0:56,11/1/2019 1:09,6.88589,79.8984,6.91373,79.8923,5.211,197.85
189125719,10.5,1087.0,80.0,0.000000,61.0,11/1/2019 1:08,11/1/2019 1:26,6.90839,79.8651,6.93669,79.9146,9.637,301.64
189127273,10.5,598.0,271.0,15.663800,68.0,11/1/2019 2:27,11/1/2019 2:37,6.92570,79.8895,6.92748,79.8971,1.880,82.30
189129552,10.5,3407.0,182.0,0.000000,112.0,11/1/2019 5:38,11/1/2019 6:35,7.13402,79.8969,6.91865,79.8649,31.163,1065.02
189132829,10.5,1246.0,487.0,0.000000,133.0,11/1/2019 6:29,11/1/2019 6:49,6.84371,79.9051,6.85069,79.8624,5.600,266.62
189135103,10.5,1333.0,295.0,17.198500,212.0,11/1/2019 6:50,11/1/2019 7:12,6.90760,79.9524,6.90634,79.9042,6.809,318.05
189139296,10.5,360.0,80.0,4.664000,3.0,11/1/2019 7:00,11/1/2019 7:06,7.26706,80.6064,7.27422,80.6124,1.945,100.32
189138671,10.5,1539.0,588.0,33.986400,43.0,11/1/2019 7:02,11/1/2019 7:28,6.85137,79.9537,6.84779,79.9274,4.203,257.89


In [48]:
def direction(df):
    d_lon = df['pick_lon'] - df['drop_lon']
    d_lat = df['pick_lat'] - df['drop_lat']
    result = np.zeros(len(d_lon))
    l = np.sqrt(d_lon**2 + d_lat**2)
    result[d_lon>0] = (180/np.pi)*np.arcsin(d_lat[d_lon>0]/l[d_lon>0])
    idx = (d_lon<0) & (d_lat>0)
    result[idx] = 180 - (180/np.pi)*np.arcsin(d_lat[idx]/l[idx])
    idx = (d_lon<0) & (d_lat<0)
    result[idx] = -180 - (180/np.pi)*np.arcsin(d_lat[idx]/l[idx])
    return result

In [49]:
def FeatureEng(X,train):
    X["duration"] = X["duration"] + 1
    X["pickup_time_min"] = (pd.to_datetime(train["pickup_time"]).dt.hour)*60 + pd.to_datetime(train["pickup_time"]).dt.minute
    X["drop_time_min"] = (pd.to_datetime(train["drop_time"]).dt.hour)*60 + pd.to_datetime(train["drop_time"]).dt.minute
    
    X["pickup_time"] = pd.to_datetime(train["pickup_time"]).dt.hour
    X["drop_time"] = pd.to_datetime(train["drop_time"]).dt.hour
    X["weekday"] = pd.to_datetime(train["pickup_time"]).dt.weekday
    
    X["avg_speed"] = (X["distance"]/(X["duration"])) * 3600
    
    X["raw_fare"] = X["fare"] -  X["additional_fare"] - X["meter_waiting_fare"]
    X["fare_per_km"] = (X["fare"]-X["meter_waiting_fare"] - X["additional_fare"])/X["distance"]
    X["fare_per_min"] = (X["fare"]/(X["duration"]))*60
    
    X["direction"] = direction(X)
 
    
    return X

In [50]:
X_train = FeatureEng(X_train,train)

/home/isuru/anaconda3/envs/Data-science/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [52]:
X_test = FeatureEng(test,test)

In [54]:
y_train = pd.DataFrame(y_train)

In [57]:
X_train.columns

Index(['additional_fare', 'duration', 'meter_waiting', 'meter_waiting_fare',
       'meter_waiting_till_pickup', 'pickup_time', 'drop_time', 'pick_lat',
       'pick_lon', 'drop_lat', 'drop_lon', 'distance', 'fare',
       'pickup_time_min', 'drop_time_min', 'weekday', 'avg_speed', 'raw_fare',
       'fare_per_km', 'fare_per_min', 'direction'],
      dtype='object')

In [58]:
X_train.to_csv("X_train_f.csv")
X_test.to_csv("X_test_f.csv")
y_train.to_csv("y_train.csv")